## Phase 1

### Header
#### Course Name: CSE487
#### phase #: 1
#### term project name: Core Trends Analysis


### Q Write down the following information: 
#### 1. Identify problems prevalent in public application domains. The issue(s) you are trying to address: This will be a 100-word abstract in the form of a problem statement.: 
    
Our goal is to research and find correlations between modern day technology users and their demographics. Our project focuses on understanding the current patterns that different groups tend to follow. This is an important issue because it shows what interests the younger and older generations are looking towards. This will help companies decide what they should focus on building towards because groups tend to follow trends. If people follow trends and companies develop products and ideas of those trends they will be able to exponentially grow their companie with their product and ideas.

Data Sources: https://www.pewresearch.org/

#### 2. Name of your data: Core Trends Analysis

#### 3. Background:
   + Source: https://www.pewresearch.org/
   + Time Period: 2011-2019
   + Scope (if there is please specified): Americans
   + Data Unit: Smartphone owners


#### 4 Introduction of dataset
Answer 2 questions:
+ How many columns? Show all the column labels. ((You can use pandas.DataFrame.columns to easily show all the labels of columns.)
    
    There are 74 columns in the dataframe
+ What are they refering to respectively? please write in table form (markdown syntax for drawing tables : [markdown tutorial with examples](https://guides.github.com/features/mastering-markdown/))

Column Header | Description
------------ | -------------
respid | This is the response id which simply is used to identify each response.
sample | This is the sample group that this response belongs to.
comp | This is whether or not the survey is complete.
int_date | This is the day the survey was taken.
lang | This is the language the survey was given in.
cregion | This is the region the survey participant was in.
state | This is the state the survey participant was in.
density | This is the county-level population density of the participant.
sfips | This is the state the survey participant was in.
usr | This is whether the participant is in an urban, suburban or rural area.
qs1 | This question asks whether the participant is under or over 18 years old.
sex | This asks for the participant's sex.
**Question codes and their respective questions** | -------------
eminuse | Do you use the internet or email, at least occasionally?
intmob | Do you access the internet on a cell phone, tablet or other mobile handheld device, at least occasionally?
intfreq | About how often do you use the internet?
snsint2 | Do you ever use social media sites like Facebook, Twitter or Instagram?
home4nw | Do you currently subscribe to internet service at HOME?
bbhome1 | Do you subscribe to dial-up internet service at home... OR do you subscribe to a higher-speed broadband service such as DSL, cable, or fiber optic service? 
bbhome2 | Just to confirm, you use a dial-up connection to the internet at home, and not a higher-speed broadband connection? 
device1a | Next, do you have a cell phone, or not?
smart2 | Is your cell phone a smartphone, or not?
q20 | Overall, when you use the internet, do you do that mostly using your cell phone or mostly using some other device like a desktop, laptop or tablet computer? 
bbsmart1 | You said that you do not have high-speed internet service at home. Did you EVER at some point in the past have a broadband internet subscription at home?
bbsmart2 | Would you like to have high-speed internet at home , or is that not something you're interested in?
**Please tell me whether any of the following are reasons why you do not have high-speed internet at home.** | -------------
bbsmart3a | The monthly cost of a home broadband subscription is too expensive
bbsmart3b | The cost of a computer is too expensive
bbsmart3c | Your smartphone lets you do everything online that you need to do
bbsmart3d | You have other options for internet access outside of your home
bbsmart3e | Broadband service is not available where you live, or is not available at an acceptable speed
bbsmart3f | Some other reason I haven’t already mentioned
bbsmart3foe@ | Some other reason I haven’t already mentioned (SPECIFY)
bbsmart4 | Thinking of the reasons why you do not have broadband service at home, which of them is the MOST important?
**Please tell me if you ever use any of the following social media sites. Do you ever use...** | -------------
web1a | Twitter
web1b | Instagram
web1c | Facebook
web1d | Snapchat
web1e | YouTube
web1f | WhatsApp
web1g | Pinterest
web1h | LinkedIn
web1i | Reddit
**Thinking about the social media sites you use... About how often do you visit or use ** | -------------
sns2a | Twitter
sns2b | Instagram
sns2c | Facebook
sns2d | Snapchat
sns2e | YouTube
**Please tell me if you happen to have each of the following items, or not. Do you have...?** | -------------
device1b | A tablet computer 
device1c | A desktop or laptop computer 
device1d | A game console 
books1 | During the past 12 months, about how many BOOKS did you read either all or part of the way through? Please include any print, electronic, or audiobooks you may have read or listened to.
**Thinking about all of the books you have read in the past 12 months, were any of those... ** | -------------
books2a | Printed books
books2b | Audiobooks
books2c | E-books
age | This asks for the participant's age.
marital | This asks for the participant's marital status.
educ2 | This asks for the participant's highest level of education.
emplnw | This asks for the participant's employment status.
hisp | This asks if the participant is hispanic.
racem1 | This asks for the participant's race.
racem2 | This asks for the participant's race. (If more than one)
racem3 | This asks for the participant's race. (If more than two)
racem4 | This asks for the participant's race. (If more than three)
racecmb | This asks for the participant's race if not their race is not included.
birth_hisp | This asks for the participant's place of birth.
inc | This asks for the participant's total family income.
party | This asks for the participant's political party.
partyln | This asks for the participant's political leaning.
hh1 | This asks for the participant's household size.
hh3 | This asks for the participant's house adult count.
ql1 | Do you have a working cell phone?
ql1a | Does anyone in your household have a working cell phone?
qc1 | Now thinking about your telephone use… Is there at least one telephone INSIDE your home that is currently working and is not a cell phone?
weight | This determines how much this response is weighed with respect to their demographics and additional calculations to reduce bias.
cellweight | This weighs their response based on if they have a cell phone or landline or both.

#### 5 Hypothesis and conclusion
+ What hypothesis or conclusion do you want to study from the data?  (Remember this relative to your future phases of task):
    
    + Model data based on age, race and sex/gender 
    + Find Trends in social media
    + Hypothesize there will be significant group that companies will be able to correlate their business with

#### 6 Write codes to load the data 
(recommend: pandas package: 10 mins tutorial: [10 mins tutorial](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html#min) )

In [680]:
import numpy as np
import pandas as pd

df = pd.read_csv('data/January 8-February 7, 2019 - Core Trends Survey - CSV.csv', header=0)

#### 7 IDA (Initial Data Analysis) 
[reference to do IDA](https://www.sciencedirect.com/science/article/pii/S0022522315017948)  \\
Please specify what you've done (text and code) for each operation (at least 10 operations). If some steps will produce a picture, please show this picture


##### 1
I want to reset the respid column and in place add values 1-(row numbers) corresponding to the correct respid

In [681]:
# code
for x in range(len(df)):
    df.loc[x,'respid'] = x

      respid  sample  comp  int_date  lang  cregion  state  density  sfips  \
0          0       1     1    190108     1        3     37        2  37069   
1          1       1     1    190108     1        2     39        3  39025   
2          2       1     1    190108     1        1     34        4  34025   
3          3       1     1    190108     1        2     18        3  18039   
4          4       1     1    190114     1        2     26        2  26027   
5          5       1     1    190108     1        1     34        5  34017   
6          6       1     1    190108     1        1     36        5  36119   
7          7       1     1    190115     1        3     47        2  47073   
8          8       1     1    190108     1        2     26        1  26033   
9          9       1     1    190108     1        2     26        3  26065   
10        10       1     1    190108     1        3     37        2  37057   
11        11       1     1    190108     1        3     21      

#### 2
I want to get rid of the bbsmart3foe@ column because it specifies other reasons why users don't have internet and is not really helpful as the information is in text and needs much work to be formatted. The bbsmart3f column already has enough details and bbsmart3foe@ is unhelpful.

In [682]:
# code
df = df.drop('bbsmart3foe@',1)
df = df.drop('ql1a',1)
df = df.drop('usr',1)


#### 3
I want to reformat the int_date column into a date format.

In [683]:
# code
df['int_date'] = pd.to_datetime(df['int_date'], format='%y%m%d')


#### 4
I want to use the comp column to check for complete responses and filter the data.

In [684]:
# code
df = df[df.comp != 0]


#### 5
I want to check for duplicate rows.

In [685]:
if(df.duplicated().any()):
    print(df.duplicated())
    df.drop_duplicates()
else:
    print("No duplicate rows found.")

No duplicate rows found.


#### 6
I want to check the qs1 "Under or over 18" question with the information in the age column.

In [686]:
# code
mask = (df['age'] >= 18)
df['qs1'][mask] = 2
mask = (df['age'] < 17)
df['qs1'][mask] = 1

Since pew research did not specify the weighing methodology, the weights will be redone based on sex and age using US Census data.

In [687]:
# code
sex = df['sex'].value_counts().tolist()
total = sex[0]+sex[1]
sex1 = 0.492 / (sex[0] / total)
sex2 = 0.508 / (sex[1] / total)

df['weight'][df['sex'] == 1] = sex1
df['weight'][df['sex'] == 2] = sex2

bins = [18, 25, 34, 54, 64, 100]
age = df['age'].value_counts(bins=bins, sort=False).tolist()
age1 = 0.114 / (age[0] / total)
age2 = 0.164 / (age[1] / total)
age3 = 0.334 / (age[2] / total)
age4 = 0.172 / (age[3] / total)
age5 = 0.216 / (age[4] / total)
pd.set_option('display.max_rows', 1502)


df['weight'][(df['age'] >= 18) & (df['age'] < 25)] = df['weight'][(df['age'] >= 18) & (df['age'] < 25)].apply(lambda x: x*age1)
df['weight'][(df['age'] >= 25) & (df['age'] < 35)] = df['weight'][(df['age'] >= 25) & (df['age'] < 35)].apply(lambda x: x*age2)
df['weight'][(df['age'] >= 35) & (df['age'] < 55)] = df['weight'][(df['age'] >= 35) & (df['age'] < 55)].apply(lambda x: x*age3)
df['weight'][(df['age'] >= 55) & (df['age'] < 65)] = df['weight'][(df['age'] >= 55) & (df['age'] < 65)].apply(lambda x: x*age4)
df['weight'][(df['age'] >= 65)] = df['weight'][(df['age'] >= 65)].apply(lambda x: x*age5)

#### 8
I want to fill out the empty hh3 column using data from the age column and the hh1 column.

In [688]:
df['hh3'][(df['age'] >= 18) & (df['hh1'] == 1) & (df['hh3'] == ' ')] = 1

#### 9
I want to check that the web columns are non-empty.

In [689]:
# code

if((df['web1a'] == ' ').any()):
    df = df[~(df['web1a'] == ' ')]
if((df['web1b'] == ' ').any()):
    df = df[~(df['web1b'] == ' ')]
if((df['web1c'] == ' ').any()):
    df = df[~(df['web1c'] == ' ')]
if((df['web1d'] == ' ').any()):
    df = df[~(df['web1d'] == ' ')]
if((df['web1e'] == ' ').any()):
    df = df[~(df['web1e'] == ' ')]
if((df['web1f'] == ' ').any()):
    df = df[~(df['web1f'] == ' ')]
if((df['web1g'] == ' ').any()):
    df = df[~(df['web1g'] == ' ')]
if((df['web1h'] == ' ').any()):
    df = df[~(df['web1h'] == ' ')]
if((df['web1i'] == ' ').any()):
    df = df[~(df['web1i'] == ' ')]

#### 10
I want to use column ql1 to fill out column device1a as it is incomplete then get rid of ql1 because it is a duplicate.

In [690]:
# code
df['device1a'][(df['ql1'] == ' ')] = 1
df = df.drop('ql1', 1)

8. (option)Remeber to save your dataSet after processing for future usage:


### Submission guideline:
1. write your code and report in this notebook. 
2. save this notebook as <person1ubname_person2ubname.ipynb>.
3. save your raw data in data folder.
4. save the notebook file with data folder in a folder named <Jiayi_person1ubname_person2ubname.zip> or <Ping_person1ubname_person2ubname.zip> or <Chen_person1ubname_person2ubname.zip> depended on which TA you have choosed. 
5. upload your zip file to timberlake server.

### Grading:
    0 – no submission by deadline
    15 – completed submission with all the requirements
    10 – there are missing items, or unsatisfactory in some respects
    5 – submission present but many required items missing or code not working
    
Once the deadline is past, you get a zero for the phase if you did not submit anything, even though you need to complete the phase to work on the next phase.


### Example Answer
(I am using the dataset from https://www.kaggle.com/gpreda/covid-world-vaccination-progress to provide a sample answer )

1. Source of Data: https://ourworldindata.org/
2. Give a name to your dataset: COVID-19 WVP (World Vaccination Progress)
3. Background:
   + Data were collected by Our World in Data
   + Time Period: 2020-2021
   + Scope (if there is please specified): Global
   + Data Unit: Individual country


|Column label|description|
|-|-|
| Country | this is the country for which the vaccination information is provided |
| Country ISO Code | ISO code for the country|
|Date | date for the data entry; for some of the dates we have only the daily vaccinations, for others, only the (cumulative) total|
|Total number of vaccinations| this is the absolute number of total immunizations in the country |
|Total number of people vaccinated| a person, depending on the immunization scheme, will receive one or more (typically 2) vaccines; at a certain moment, the number of vaccination might be larger than the number of people|
|Total number of people fully vaccinated| the number of people that received the entire set of immunization according to the immunization scheme (typically 2); at a certain moment in time, there might be a certain number of people that received one vaccine and another number (smaller) of people that received all vaccines in the scheme|
|Daily vaccinations (raw) | for a certain data entry, the number of vaccination for that date/country|
|Daily vaccinations | for a certain data entry, the number of vaccination for that date/country|
|Total vaccinations per hundred| ratio (in percent) between vaccination number and total population up to the date in the country|
|Total number of people vaccinated per hundred | ratio (in percent) between population immunized and total population up to the date in the country|
|Total number of people fully vaccinated per hundred |ratio (in percent) between population fully immunized and total population up to the date in the country|
|Number of vaccinations per day | number of daily vaccination for that day and country|
|Daily vaccinations per million | ratio (in ppm) between vaccination number and total population for the current date in the country|
|Vaccines used in the country | total number of vaccines used in the country (up to date)|
|Source name | source of the information (national authority, international organization, local organization etc.)|
|Source website | website of the source of information|

4. Introduction of dataset: Answer 2 questions:
+ How many columns? Show all the column labels. ((You can use pandas.DataFrame.columns to easily show all the labels of columns.)
+ What are they refering to respectively? please write in table form (markdown syntax for drawing tables : [markdown tutorial with examples](https://guides.github.com/features/mastering-markdown/))

5. What hypothesis or conclusion do you want to study from the data?  (Remember this relative to your future phases of task)\\
The relation between time with respect to increaseing of vaccinated people per day is decreasing negative exponetially (You don't need to be too specific on the hypothesis, something vague is fine. But the hypothesis mustn't be too obvious to analysis, e.g. find the country with largest vaccinated people ). \\

6. Write codes to load the data (recommend: pandas package) save your data in your google drive "./data" folder

In [691]:
import pandas as pd
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/My Drive/data/test.csv")

ModuleNotFoundError: No module named 'google'

7. IDA (Initial Data Analysis)

Statement: I want to take a look at the brief description of data



```
df.describe()
```


8. (option)Remeber to save your dataSet after processing for future usage:


In [ ]:
df.to_csv(index=False) # for example